In [75]:
import tensorflow
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K

In [76]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [77]:
print(x_train.shape, y_train.shape)


(60000, 28, 28) (60000,)


In [78]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)

In [79]:
# convert class vectors to binary class matrices
y_train =tensorflow.keras.utils.to_categorical(y_train, 10)
y_test = tensorflow.keras.utils.to_categorical(y_test, 10)


In [80]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [81]:
batch_size = 228
num_classes = 10
epochs = 60

model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5),activation='relu',input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))


In [82]:
model.compile(loss=tensorflow.keras.losses.categorical_crossentropy,optimizer=tensorflow.keras.optimizers.Adadelta(),metrics=['accuracy'])


In [83]:
hist = model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(x_test, y_test))
print("The model has successfully trained")

Epoch 1/60
264/264 [==============================] - 53s 196ms/step - loss: 2.3152 - accuracy: 0.1085 - val_loss: 2.3023 - val_accuracy: 0.1347
Epoch 2/60
264/264 [==============================] - 36s 136ms/step - loss: 2.3041 - accuracy: 0.1169 - val_loss: 2.2921 - val_accuracy: 0.1602
Epoch 3/60
264/264 [==============================] - 24s 92ms/step - loss: 2.2954 - accuracy: 0.1283 - val_loss: 2.2825 - val_accuracy: 0.1967
Epoch 4/60
264/264 [==============================] - 22s 83ms/step - loss: 2.2867 - accuracy: 0.1431 - val_loss: 2.2732 - val_accuracy: 0.2516
Epoch 5/60
264/264 [==============================] - 22s 84ms/step - loss: 2.2789 - accuracy: 0.1549 - val_loss: 2.2643 - val_accuracy: 0.3079
Epoch 6/60
264/264 [==============================] - 21s 79ms/step - loss: 2.2696 - accuracy: 0.1709 - val_loss: 2.2554 - val_accuracy: 0.3473
Epoch 7/60
264/264 [==============================] - 22s 82ms/step - loss: 2.2621 - accuracy: 0.1820 - val_loss: 2.2461 - val_accurac

264/264 [==============================] - 21s 80ms/step - loss: 1.3239 - accuracy: 0.5742 - val_loss: 0.9837 - val_accuracy: 0.8086
Epoch 58/60
264/264 [==============================] - 21s 81ms/step - loss: 1.3111 - accuracy: 0.5777 - val_loss: 0.9646 - val_accuracy: 0.8131
Epoch 59/60
264/264 [==============================] - 21s 80ms/step - loss: 1.2940 - accuracy: 0.5860 - val_loss: 0.9442 - val_accuracy: 0.8169
Epoch 60/60
264/264 [==============================] - 20s 77ms/step - loss: 1.2779 - accuracy: 0.5874 - val_loss: 0.9252 - val_accuracy: 0.8219
The model has successfully trained


In [84]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.9252374768257141
Test accuracy: 0.8219000101089478


In [85]:
model.save('mnist2.h5')
print("Saving the model as mnist.h5")

Saving the model as mnist.h5


In [86]:
from keras.models import load_model
from tkinter import *
import tkinter as tk
import win32gui
from PIL import ImageGrab, Image
import numpy as np

In [87]:
model = load_model('mnist2.h5')


In [88]:
def predict_digit(img):
    #resize image to 28x28 pixels
    img = img.resize((28,28))
    #convert rgb to grayscale
    img = img.convert('L')
    img = np.array(img)
    #reshaping to support our model input and normalizing
    img = img.reshape(1,28,28,1)
    img = img/255.0
    #predicting the class
    res = model.predict([img])[0]
    return np.argmax(res), max(res)


In [89]:
class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)

        self.x = self.y = 0
        
        # Creating elements
        self.canvas = tk.Canvas(self, width=300, height=300, bg = "white", cursor="cross")
        self.label = tk.Label(self, text="Draw..", font=("Helvetica", 48))
        self.classify_btn = tk.Button(self, text = "Recognise", command = self.classify_handwriting)   
        self.button_clear = tk.Button(self, text = "Clear", command = self.clear_all)
       
        # Grid structure
        self.canvas.grid(row=0, column=0, pady=2, sticky=W, )
        self.label.grid(row=0, column=1,pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)
        
        #self.canvas.bind("<Motion>", self.start_pos)
        self.canvas.bind("<B1-Motion>", self.draw_lines)

    def clear_all(self):
        self.canvas.delete("all")
        
    def classify_handwriting(self):
        HWND = self.canvas.winfo_id()  # get the handle of the canvas
        rect = win32gui.GetWindowRect(HWND)  # get the coordinate of the canvas
        a,b,c,d = rect
        rect=(a+4,b+4,c-4,d-4)
        im = ImageGrab.grab(rect)

        digit, acc = predict_digit(im)
        self.label.configure(text= str(digit)+', '+ str(int(acc*100))+'%')

    def draw_lines(self, event):
        self.x = event.x
        self.y = event.y
        r=8
        self.canvas.create_oval(self.x-r, self.y-r, self.x + r, self.y + r, fill='black')
       

In [90]:
app = App()
mainloop()
